In [1]:
import numpy as np
import pandas as pd
import torch
import random

random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
train = pd.read_csv('x_train.csv',encoding='CP949')
test = pd.read_csv('x_test.csv',encoding='CP949')
train_label = pd.read_csv('y_train.csv',encoding='CP949')
submit = pd.read_csv('sample_submission.csv',encoding='CP949')

In [3]:
train

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,월,일,년
0,0,26,76018.965,5009.811,36887.341,117916.117,40.467,21.667,17.750,0.000,2.725,9,25,2019
1,1,24,1232416.968,76043.955,451558.268,1760019.191,38.126,31.632,13.300,14.500,4.500,3,4,2018
2,1,23,134260.946,8412.761,78177.981,220851.688,51.107,45.643,5.250,0.500,4.950,4,7,2018
3,1,25,289327.429,39102.424,287029.303,615459.156,33.067,48.000,26.300,0.500,1.500,8,14,2019
4,1,7,1116617.660,96560.651,524905.301,1738083.612,40.392,26.961,8.400,44.000,4.300,12,1,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21481,0,6,91977.598,14802.819,79885.446,186665.863,41.676,21.676,6.400,0.000,1.600,3,5,2020
21482,0,9,63673.610,3725.663,65813.385,133212.658,48.737,26.105,18.300,7.000,3.700,5,26,2018
21483,1,3,406035.642,46335.550,258079.137,710450.329,49.314,50.943,-6.133,0.833,6.067,1,25,2018
21484,1,25,315737.894,25379.224,283079.548,624196.666,35.312,43.688,8.600,0.000,3.100,3,17,2019


In [4]:
train_x = train.drop(['읍면동명','일'], axis = 1) #해당 레이블은 각각 index. label 이므로 제거
test_x = test.drop(['읍면동명', '일'], axis = 1)
train_y = train_label

In [5]:
#from sklearn.preprocessing import StandardScaler

#sc = StandardScaler()
#train_x_sc = sc.fit_transform(train_x)
#test_x_sc= sc.transform(test_x)

In [6]:
#train_x_tensor = torch.FloatTensor(np.array(train_x)).to(device)
#test_x_tensor = torch.FloatTensor(np.array(test_x)).to(device)
train_x_tensor = torch.FloatTensor(train_x.values).to(device)
test_x_tensor = torch.FloatTensor(test_x.values).to(device)

train_y_tensor = torch.FloatTensor(train_y.values).to(device)

In [7]:
train_dataset = torch.utils.data.TensorDataset(train_x_tensor, train_y_tensor)
data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=1000,
                                          shuffle=True,
                                          drop_last=True)

In [8]:
linear1 = torch.nn.Linear(12,512,bias=True)
linear2 = torch.nn.Linear(512,1024,bias=True)
linear3 = torch.nn.Linear(1024,2048,bias=True)
linear4 = torch.nn.Linear(2048,1024,bias=True)
linear5 = torch.nn.Linear(1024,1024,bias=True)
linear6 = torch.nn.Linear(1024,512,bias=True)
linear7 = torch.nn.Linear(512,512,bias=True)
linear8 = torch.nn.Linear(512,1,bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.3)

In [9]:
# Random Init => Xavier Init
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)
torch.nn.init.xavier_normal_(linear5.weight)
torch.nn.init.xavier_normal_(linear6.weight)
torch.nn.init.xavier_normal_(linear7.weight)
torch.nn.init.xavier_normal_(linear8.weight)

Parameter containing:
tensor([[-0.0285, -0.0580,  0.1168,  0.0281,  0.0467,  0.0602, -0.0461,  0.0095,
          0.0528,  0.0024, -0.0776,  0.0151, -0.1371, -0.0689,  0.1509,  0.0739,
          0.0316, -0.0106, -0.1320, -0.0720, -0.0695, -0.0340,  0.1161,  0.0082,
          0.0199, -0.0375, -0.0632, -0.0661,  0.0277, -0.0169, -0.0715, -0.0691,
          0.0257,  0.1109, -0.0229, -0.0382, -0.0093, -0.0456,  0.0451,  0.0258,
         -0.1519, -0.0570, -0.0199,  0.0236,  0.0803, -0.0064, -0.0117,  0.0326,
         -0.0663,  0.0404,  0.0586,  0.0719, -0.0435,  0.0950,  0.0705,  0.0194,
          0.1047,  0.0912, -0.0009,  0.0712, -0.0137, -0.0108, -0.0408, -0.0473,
         -0.0821,  0.0072, -0.0283,  0.0218,  0.0622,  0.0444,  0.0113, -0.0275,
         -0.1782,  0.0997, -0.0682, -0.0350,  0.1030, -0.0629, -0.0005, -0.0299,
          0.0299,  0.0210, -0.0077,  0.0246,  0.0051, -0.0050, -0.0850, -0.0212,
          0.0203,  0.0562, -0.1291, -0.0593,  0.0324,  0.1059,  0.1097,  0.0008,
      

In [10]:
model = torch.nn.Sequential(linear1,relu,
                            linear2,relu,
                            linear3,relu,
                            linear4,relu,
                            linear5,relu,
                            linear6,relu,
                            linear7,relu,
                            linear8).to(device)

In [11]:
loss = torch.nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) 

In [12]:

total_batch = len(data_loader)
model.train() # 주의사항 drop_out = True
for epoch in range(500):
    avg_cost = 0

    for X, Y in data_loader:


        # 그래디언트 초기화
        optimizer.zero_grad()
        # Forward 계산
        hypothesis = model(X)
        # Error 계산
        cost = loss(hypothesis, Y)
        # Backparopagation
        cost.backward()
        # 가중치 갱신
        optimizer.step()

        # 평균 Error 계산
        avg_cost += cost / total_batch

    if (epoch + 1) % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished') #61154

Epoch: 0010 cost = 193340.000000000
Epoch: 0020 cost = 176634.421875000
Epoch: 0030 cost = 429804.375000000
Epoch: 0040 cost = 165898.031250000
Epoch: 0050 cost = 157923.031250000
Epoch: 0060 cost = 157559.000000000
Epoch: 0070 cost = 142521.234375000
Epoch: 0080 cost = 186590.625000000
Epoch: 0090 cost = 128077.875000000
Epoch: 0100 cost = 105411.929687500
Epoch: 0110 cost = 108129.851562500
Epoch: 0120 cost = 98306.710937500
Epoch: 0130 cost = 93624.328125000
Epoch: 0140 cost = 91833.421875000
Epoch: 0150 cost = 101232.695312500
Epoch: 0160 cost = 88441.265625000
Epoch: 0170 cost = 95710.773437500
Epoch: 0180 cost = 86506.281250000
Epoch: 0190 cost = 86782.273437500
Epoch: 0200 cost = 87214.812500000
Epoch: 0210 cost = 84635.960937500
Epoch: 0220 cost = 84733.320312500
Epoch: 0230 cost = 87508.632812500
Epoch: 0240 cost = 83733.757812500
Epoch: 0250 cost = 77532.617187500
Epoch: 0260 cost = 77983.992187500
Epoch: 0270 cost = 75063.656250000
Epoch: 0280 cost = 69491.117187500
Epoch: 0

In [13]:
from sklearn.metrics import r2_score
with torch.no_grad():
    model.eval()

    prediction = model(test_x_tensor)
    train_prediction = model(train_x_tensor)
    accuracy = r2_score(train_y.교통량, train_prediction.detach().cpu())
    print('R2:', accuracy.item())

R2: 0.7286287673884821


In [14]:
submit.predict = prediction.detach().cpu().numpy()

In [15]:
submit.to_csv('2021_10_13_1.csv', index= False)

baseline: 11,121/ 121,242 / 242, 242/ 242, 121 / 121 , 1 => -0.14

=> 진행중 score: 0.27 MSE: 257624.